Answer all questions and submit them either as an IPython notebook, LaTeX document, or Markdown document. Provide full answers for each question, including interpretation of the results. Each question is worth 25 points.

This homework is due on Monday, November 30, 2015.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np

from pandas import Series,DataFrame
#from sklearn import svm
#from sklearn.cross_validation import train_test_split
#from sklearn.linear_model import LogisticRegression
#from sklearn.base import TransformerMixin
#from sklearn import preprocessing
#from sklearn import model_selection
import datetime
from dateutil.relativedelta import relativedelta
import csv
import sys

In [3]:
adt = pd.read_csv('Data/FONNESBECK_ADT_20151202.csv',encoding='cp1252')
del adt['Event_Date']
adt_discharge = adt.iloc[adt[adt['Event']=='Discharge'].index.tolist()]
pheno = pd.read_csv("Data/FONNESBECK_phenotype_20151202.csv")
del pheno['DOD']

In [4]:
adt_pheno = pd.merge(adt_discharge, pheno, how='inner', on='RUID')
del adt_pheno['Event']
adt_pheno.Admission_date = pd.to_datetime(adt_pheno.Admission_date)
adt_pheno.DISCHARGE_DATE = pd.to_datetime(adt_pheno.DISCHARGE_DATE)
adt_pheno.DOB = pd.to_datetime(adt_pheno.DOB)
adt_pheno['AGE'] = pd.Series(np.zeros(adt_pheno.shape[0]), index=adt_pheno.index)

In [8]:
adt_phenoDOBNP = adt_pheno[['RUID','Admission_date','DOB']].values
age = np.zeros((adt_pheno.shape[0],1))
for i in range(adt_pheno.shape[0]):
    if type(adt_phenoDOBNP[i][1]) is pd.tslib.Timestamp and  type(adt_pheno.DOB[i]) is pd.tslib.Timestamp:
        rdelta = relativedelta(adt_phenoDOBNP[i][1], adt_phenoDOBNP[i][2])
        age[i] = rdelta.years
adt_pheno['AGE'] = age

In [9]:
adt_phenoNP = adt_pheno[['RUID','Admission_date','DISCHARGE_DATE']].values

In [10]:
icd9 = pd.read_csv('Data/FONNESBECK_ICD9_20151202.csv',quoting=csv.QUOTE_NONE)
icd9.Event_date = pd.to_datetime(icd9.Event_date)
icd9NP = icd9.values

In [11]:
'''
http://www.icd9data.com/2012/Volume1/default.htm
001-139  Infectious And Parasitic Diseases
140-239  Neoplasms
240-279  Endocrine, Nutritional And Metabolic Diseases, And Immunity Disorders
280-289  Diseases Of The Blood And Blood-Forming Organs
290-319  Mental Disorders
320-389  Diseases Of The Nervous System And Sense Organs
390-459  Diseases Of The Circulatory System
460-519  Diseases Of The Respiratory System
520-579  Diseases Of The Digestive System
580-629  Diseases Of The Genitourinary System
630-679  Complications Of Pregnancy, Childbirth, And The Puerperium
680-709  Diseases Of The Skin And Subcutaneous Tissue
710-739  Diseases Of The Musculoskeletal System And Connective Tissue
740-759  Congenital Anomalies
760-779  Certain Conditions Originating In The Perinatal Period
780-799  Symptoms, Signs, And Ill-Defined Conditions
800-999  Injury And Poisoning
V01-V91  Supplementary Classification Of Factors Influencing Health Status And Contact With Health Services
E000-E999  Supplementary Classification Of External Causes Of Injury And Poisoning
'''
icd9Classes = np.zeros((19,2))
icd9Classes[0][0],icd9Classes[0][1] = 0,139
icd9Classes[1][0],icd9Classes[1][1] = 140,239
icd9Classes[2][0],icd9Classes[2][1] = 240,279
icd9Classes[3][0],icd9Classes[3][1] = 280,289
icd9Classes[4][0],icd9Classes[4][1] = 290,319
icd9Classes[5][0],icd9Classes[5][1] = 320,389
icd9Classes[6][0],icd9Classes[6][1] = 390,459
icd9Classes[7][0],icd9Classes[7][1] = 460,519
icd9Classes[8][0],icd9Classes[8][1] = 520,579
icd9Classes[9][0],icd9Classes[9][1] = 580,629
icd9Classes[10][0],icd9Classes[10][1] = 630,679
icd9Classes[11][0],icd9Classes[11][1] = 680,709
icd9Classes[12][0],icd9Classes[12][1] = 710,739
icd9Classes[13][0],icd9Classes[13][1] = 740,759
icd9Classes[14][0],icd9Classes[14][1] = 760,779
icd9Classes[15][0],icd9Classes[15][1] = 780,799
icd9Classes[16][0],icd9Classes[16][1] = 800,999
icd9Classes[17][0],icd9Classes[17][1] = 1,91
icd9Classes[18][0],icd9Classes[18][1] = 0,999

In [13]:
uniqueicd9 = Series(icd9.ICD9_Code.ravel()).unique()
roundList = dict()
uniqueNameList = dict()
for i in range(uniqueicd9.shape[0]):
    myString = str(uniqueicd9[i])
    if myString == 'nan':
        continue
    indexDot = myString.find('.')
    if indexDot is not -1:
        myString = myString[:indexDot]
    if myString.find('V') == -1 and myString.find('E')==-1:
        myStringInt = int(myString)
    if myStringInt == 0:
        myString = '0'
    if uniqueicd9[i] not in roundList:
        roundList[uniqueicd9[i]] = myString
        uniqueNameList[myString] = 1
    

In [16]:
ruidDict = dict()
for i in range(len(icd9NP)):
    ruid = icd9NP[i][0]
    if ruid not in ruidDict:
        ruidList = []
        ruidList.append(i)
        ruidDict[ruid] = ruidList
    else:
        ruidList = ruidDict[ruid]
        ruidList.append(i)
        ruidDict[ruid] = ruidList

In [ ]:
#Old way
'''
time_now = datetime.datetime.now()
howManyicd9 = np.zeros((5000,1))
npICD9Count = np.zeros((adt_pheno.shape[0],len(uniqueNameList.keys())))
checkList = []
for i in range(501):#adt_pheno.shape[0]):
    if adt_phenoNP[i][0] not in ruidDict:
        print('Non existent')
        continue
    ruidIndexList = ruidDict[adt_phenoNP[i][0]]
    #print(ruidIndexList)
    counter = 0
    for j in range(len(ruidIndexList)):
        date = icd9NP[ruidIndexList[j]][2]
        start = adt_phenoNP[i][1]
        end =  adt_phenoNP[i][2]
        if start <= date <= end:
            myString = str(roundList[icd9NP[ruidIndexList[j]][1]])
            if myString == 'nan':
                continue
            npICD9Count[i][columIndex[myString]] = 1
            counter +=1
            checkList.append(columIndex[myString])
    howManyicd9[i] = counter
diff = datetime.datetime.now() - time_now
diff
for key in adt_pheno.columns.values:
    if key in columIndex.keys():
        adt_pheno[key] = npICD9Count[:,columIndex[key]]
'''

In [17]:
time_now = datetime.datetime.now()
howManyicd9 = np.zeros((adt_pheno.shape[0],1))
npICD9Count = np.zeros((adt_pheno.shape[0],len(icd9Classes)))
for i in range(501):#adt_pheno.shape[0]):
    if adt_phenoNP[i][0] not in ruidDict:
        #print('Non existent',i)
        continue
    ruidIndexList = ruidDict[adt_phenoNP[i][0]]
    counter = 0
    setPerStay = set()
    for j in range(len(ruidIndexList)):
        date = icd9NP[ruidIndexList[j]][2]
        start = adt_phenoNP[i][1]
        end =  adt_phenoNP[i][2]
        if start <= date <= end:
            vFound = False
            eFound = False
            myString = str(roundList[icd9NP[ruidIndexList[j]][1]])
            if myString == 'nan':
                continue
            if myString.find('V') != -1:
                vFound = True
            elif myString.find('E')!=-1:
                eFound = True
            whichColumn = -1
            if vFound or eFound:
                myStringInt = int(myString[1:])
                for k in range(len(icd9Classes)-2,len(icd9Classes)):
                    if icd9Classes[k][0] <= myStringInt <= icd9Classes[k][1]:
                        whichColumn = k
                        break
            else:
                myStringInt = int(myString)
                for k in range(len(icd9Classes)-2):
                    if icd9Classes[k][0] <= myStringInt <= icd9Classes[k][1]:
                        whichColumn = k
                        break
            if myString in setPerStay:
                continue
            setPerStay.add(myString)
            if whichColumn!= -1:
                npICD9Count[i][k] += 1
            else:
                print('Problem',i,myStringInt,myString)
            counter +=1
    howManyicd9[i] = counter
for k in range(len(icd9Classes)):
    adt_pheno['ICD9_' + str(k)] = pd.Series(npICD9Count[:,k], index=adt_pheno.index)
diff = datetime.datetime.now() - time_now
diff

datetime.timedelta(0, 1, 350098)

In [19]:
cpt= pd.read_csv('Data/FONNESBECK_CPT_20151202.csv',quoting=csv.QUOTE_NONE)
cpt.Event_date = pd.to_datetime(cpt.Event_date)
cptNP = cpt.values

In [20]:
'''
https://en.wikipedia.org/wiki/Current_Procedural_Terminology#cite_note-8
Codes for surgery: 10000–69990
(10000–10022) general
(10040–19499) integumentary system
(20000–29999) musculoskeletal system
(30000–32999) respiratory system
(33010–37799) cardiovascular system
(38100–38999) hemic and lymphatic systems
(39000–39599) mediastinum and diaphragm
(40490–49999) digestive system
(50010–53899) urinary system
(54000–55899) male genital system
(55920–55980) reproductive system and intersex
(56405–58999) female genital system
(59000–59899) maternity care and delivery
(60000–60699) endocrine system
(61000–64999) nervous system
(65091–68899) eye and ocular adnexa
(69000–69979) auditory system
Codes for Radiology: 70000-79999[edit]
(70000–76499) diagnostic imaging
(76506–76999) diagnostic ultrasound
(77001–77032) radiologic guidance
(77051–77059) breast mammography
(77071–77084) bone/joint studies
(77261–77799) radiation oncology
(78000–79999) nuclear medicine
Codes for pathology and laboratory: 80000–89398[edit]
(80000–80076) organ or disease-oriented panels
(80100–80103) drug testing
(80150–80299) therapeutic drug assays
(80400–80440) evocative/suppression testing
(80500–80502) consultations (clinical pathology)
(81000–81099) urinalysis
(82000–84999) chemistry
(85002–85999) hematology and coagulation
(86000–86849) immunology
(86850–86999) transfusion medicine
(87001–87999) microbiology
(88000–88099) anatomic pathology (postmortem)
(88104–88199) cytopathology
(88230–88299) cytogenetic studies
(88300–88399) surgical pathology
(88720–88741) in vivo (transcutaneous) lab procedures
(89049–89240) other procedures
(89250–89398) reproductive medicine procedures
Codes for medicine: 90281–99099; 99151–99199; 99500–99607[edit]
(90281–90399) immune globulins, serum or recombinant prods
(90465–90474) immunization administration for vaccines/toxoids
(90476–90749) vaccines, toxoids
(90801–90899) psychiatry
(90901–90911) biofeedback
(90935–90999) dialysis
(91000–91299) gastroenterology
(92002–92499) ophthalmology
(92502–92700) special otorhinolaryngologic services
(92950–93799) cardiovascular
(93875–93990) noninvasive vascular diagnostic studies
(94002–94799) pulmonary
(95004–95199) allergy and clinical immunology
(95250–95251) endocrinology
(95803–96020) neurology and neuromuscular procedures
(96101–96125) central nervous system assessments/tests (neuro-cognitive, mental status, speech testing)
(96150–96155) health and behavior assessment/intervention
(96360–96549) hydration, therapeutic, prophylactic, diagnostic injections and infusions, and chemotherapy and other highly complex drug or highly complex biologic agent administration
(96567–96571) photodynamic therapy
(96900–96999) special dermatological procedures
(97001–97799) physical medicine and rehabilitation
(97802–97804) medical nutrition therapy
(97810–97814) acupuncture
(98925–98929) osteopathic manipulative treatment
(98940–98943) chiropractic manipulative treatment
(98960–98962) education and training for patient self-management
(98966–98969) non-face-to-face nonphysician services
(99000–99091) special services, procedures and reports
(99170–99199) other services and procedures
(99500–99602) home health procedures/services
(99605–99607) medication therapy management services
'''

cptClassesGeneral = np.zeros((4,2))
cptClassesGeneral[0][0],cptClassesGeneral[0][1] = 10000,69990
cptClassesGeneral[1][0],cptClassesGeneral[1][1] = 70000,79999
cptClassesGeneral[2][0],cptClassesGeneral[2][1] = 80000,89398
cptClassesGeneral[3][0],cptClassesGeneral[3][1] = 90281,99607

cptClassesSurgery = np.zeros((15,2))
cptClassesSurgery[0][0],cptClassesSurgery[0][1] = 10000,10022
cptClassesSurgery[1][0],cptClassesSurgery[1][1] = 10040,19499
cptClassesSurgery[2][0],cptClassesSurgery[2][1] = 20000,29999
cptClassesSurgery[3][0],cptClassesSurgery[3][1] = 30000,32999
cptClassesSurgery[4][0],cptClassesSurgery[4][1] = 33010,37799
cptClassesSurgery[5][0],cptClassesSurgery[5][1] = 38100,38999
cptClassesSurgery[6][0],cptClassesSurgery[6][1] = 39000,39599
cptClassesSurgery[7][0],cptClassesSurgery[7][1] = 40490,49999
cptClassesSurgery[8][0],cptClassesSurgery[8][1] = 50010,53899
cptClassesSurgery[9][0],cptClassesSurgery[9][1] = 54000,55899
cptClassesSurgery[10][0],cptClassesSurgery[10][1] = 56405,58999
cptClassesSurgery[11][0],cptClassesSurgery[11][1] = 59000,59899
cptClassesSurgery[12][0],cptClassesSurgery[12][1] = 60000,64999
cptClassesSurgery[13][0],cptClassesSurgery[13][1] = 65091,68899
cptClassesSurgery[14][0],cptClassesSurgery[14][1] = 69000,69979


In [21]:
'''
2011 HCPCS Codes
http://www.icd9data.com/HCPCS/2011/default.htm
A codes
B codes
C codes
E codes
G codes
H codes
J codes
K codes
L codes
M codes
P codes
Q codes
R codes
S codes
T codes
V codes
'''
hcpcs = np.zeros((16,1))
hcpcs[0] = ord('A')
hcpcs[1] = ord('B')
hcpcs[2] = ord('C')
hcpcs[3] = ord('E')
hcpcs[4] = ord('G')
hcpcs[5] = ord('H')
hcpcs[6] = ord('J')
hcpcs[7] = ord('K')
hcpcs[8] = ord('L')
hcpcs[9] = ord('M')
hcpcs[10] = ord('P')
hcpcs[11] = ord('Q')
hcpcs[12] = ord('R')
hcpcs[13] = ord('S')
hcpcs[14] = ord('T')
hcpcs[15] = ord('V')

In [22]:
def get_num(x):
    return int(''.join(ele for ele in x if ele.isdigit()))

In [23]:
ruidDict = dict()
for i in range(len(cptNP)):
    ruid = cptNP[i][0]
    if ruid not in ruidDict:
        ruidList = []
        ruidList.append(i)
        ruidDict[ruid] = ruidList
    else:
        ruidList = ruidDict[ruid]
        ruidList.append(i)
        ruidDict[ruid] = ruidList

In [25]:
time_now = datetime.datetime.now()
npCPTCount = np.zeros((adt_pheno.shape[0],len(cptClassesSurgery)+len(cptClassesGeneral)+len(hcpcs)))
for i in range(adt_pheno.shape[0]):
    if i % 5000 == 0:
        print(i)
    if adt_phenoNP[i][0] not in ruidDict:
        #print('Non existent',i)
        continue
    ruidIndexList = ruidDict[adt_phenoNP[i][0]]
    counter = 0
    setPerStay = set()
    for j in range(len(ruidIndexList)):
        date = cptNP[ruidIndexList[j]][2]
        start = adt_phenoNP[i][1]
        end =  adt_phenoNP[i][2]
        if start <= date <= end:
            vFound = False
            eFound = False
            cptCode = cptNP[j][1]
            if cptCode in setPerStay:
                continue
            setPerStay.add(cptCode)
            surgicalFound = False 
            #First general then surgical then hcpcs
            if get_num(cptCode) > 10000:
                for k in range(len(cptClassesGeneral)):
                    if cptClassesGeneral[k][0] <= get_num(cptCode) <= cptClassesGeneral[k][1]:
                        npCPTCount[i][k] +=1
                        if k ==0:
                            surgicalFound = True
                        break
            else:
                for k in range(len(hcpcs)):
                    if hcpcs[k] == ord(cptCode[0]):
                        npCPTCount[i][len(cptClassesGeneral)+len(cptClassesSurgery)+k] +=1
                        break
            if surgicalFound:
                for k in range(len(cptClassesSurgery)):
                    if cptClassesSurgery[k][0] <= get_num(cptCode) <= cptClassesSurgery[k][1]:
                        npCPTCount[i][len(cptClassesGeneral)+k] +=1
                        break
            #print(cptCode)
            counter +=1
    howManyicd9[i] = counter
for k in range(len(cptClassesGeneral)+len(cptClassesSurgery)+len(hcpcs)):
    if k < len(cptClassesGeneral):
        adt_pheno['CPT_G_' + str(k)] = pd.Series(npCPTCount[:,k], index=adt_pheno.index)
    elif k >= len(cptClassesGeneral) and k < len(cptClassesGeneral)+len(cptClassesSurgery):
        adt_pheno['CPT_S_' + str(k-4)] = pd.Series(npCPTCount[:,k], index=adt_pheno.index)
    else:
        adt_pheno['HCPCS_S_' + str(k-19)] = pd.Series(npCPTCount[:,k], index=adt_pheno.index)
diff = datetime.datetime.now() - time_now
diff

datetime.timedelta(0, 84, 795428)

In [27]:
adt_phenoSorted = adt_pheno.sort(columns='DISCHARGE_DATE')
adt_phenoSortedNP = adt_phenoSorted[['RUID','Admission_date','DISCHARGE_DATE']].values

In [28]:
adt_pheno['lastVisitInDays'] = pd.Series(np.zeros(adt_pheno.shape[0]), index=adt_pheno.index)
adt_pheno['readmissionIn30'] = pd.Series(np.zeros(adt_pheno.shape[0]), index=adt_pheno.index)
adt_pheno['numberOfVisitsIn6Months'] = pd.Series(np.zeros(adt_pheno.shape[0]), index=adt_pheno.index)
adt_pheno['readmittedIn30'] = pd.Series(np.zeros(adt_pheno.shape[0]), index=adt_pheno.index)

In [29]:
ruidDict = dict()
for i in range(len(adt_phenoSortedNP)):
    ruid = adt_phenoSortedNP[i][0]
    if ruid not in ruidDict:
        ruidList = []
        ruidList.append(i)
        ruidDict[ruid] = ruidList
    else:
        ruidList = ruidDict[ruid]
        ruidList.append(i)
        ruidDict[ruid] = ruidList

In [55]:
time_now = datetime.datetime.now()
lastVisitInDays = np.zeros((adt_pheno.shape[0],1))
readmissionIn30 = np.zeros((adt_pheno.shape[0],1))
numberOfVisitsIn6Months = np.zeros((adt_pheno.shape[0],1))
readmittedIn30 = np.zeros((adt_pheno.shape[0],1))
for i in range(adt_pheno.shape[0]):
    if i % 1000 == 0:
        print(i)
    if adt_phenoSortedNP[i][0] not in ruidDict:
        #print('Non existent',i)
        continue
    ruidIndexList = ruidDict[adt_phenoSortedNP[i][0]]
    closestToI = -1
    for j in range(len(ruidIndexList)):
        if j > 0:
            date = adt_phenoSortedNP[ruidIndexList[j]][1] - adt_phenoSortedNP[ruidIndexList[j-1]][2]
            if type(date) is pd.tslib.Timedelta:
                if date.days < 30:
                    readmissionIn30[i] +=1
        if ruidIndexList[j] >= i:
            break
        if ruidIndexList[j] > closestToI:
            closestToI = ruidIndexList[j]
        date = adt_phenoSortedNP[ruidIndexList[j]][2]
        start =  adt_phenoSortedNP[i][1]
        if date > start:
            print('Skip')
            continue
        daysPassed = start-date
        if type(daysPassed) is pd.tslib.Timedelta:
            daysPassed = daysPassed.days
            if daysPassed < 180:
                numberOfVisitsIn6Months[i] +=1
    if closestToI>-1:
        date = adt_phenoSortedNP[i][1] - adt_phenoSortedNP[closestToI][2]
        if type(date) is pd.tslib.Timedelta:
            lastVisitInDays[i] = date.days
    if ruidIndexList[j] != i:
        print('Skip')
        continue
    if j == len(ruidIndexList)-1:
        continue
    date = adt_phenoSortedNP[ruidIndexList[j+1]][1] - adt_phenoSortedNP[ruidIndexList[j]][2]
    if type(date) is pd.tslib.Timedelta:
        if date.days < 30:
            readmittedIn30[i] = 1
adt_pheno['lastVisitInDays'] = lastVisitInDays
adt_pheno['readmissionIn30'] = readmissionIn30
adt_pheno['numberOfVisitsIn6Months'] = numberOfVisitsIn6Months
adt_pheno['readmittedIn30'] = readmittedIn30
diff = datetime.datetime.now() - time_now
diff

0
1000
2000
3000
4000
5000
Skip
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
Skip
19000
20000
21000
22000
23000
24000
25000
Skip
Skip
26000
Skip
Skip
Skip
Skip
Skip
27000
Skip
28000
Skip
Skip
Skip
29000
Skip
Skip


datetime.timedelta(0, 19, 885183)

In [82]:
bp = pd.read_csv('Data/FONNESBECK_BP_20151202.csv',quoting=csv.QUOTE_NONE)
bpNP = bp.values

In [83]:
ruidDict = dict()
for i in range(len(bpNP)):
    ruid = bpNP[i][0]
    if ruid not in ruidDict:
        ruidList = []
        ruidList.append(i)
        ruidDict[ruid] = ruidList
    else:
        ruidList = ruidDict[ruid]
        ruidList.append(i)
        ruidDict[ruid] = ruidList

In [84]:
adt_pheno['meanSys'] = pd.Series(np.zeros(adt_pheno.shape[0]), index=adt_pheno.index)
adt_pheno['diffSys'] = pd.Series(np.zeros(adt_pheno.shape[0]), index=adt_pheno.index)
adt_pheno['meanDias'] = pd.Series(np.zeros(adt_pheno.shape[0]), index=adt_pheno.index)
adt_pheno['diffDias'] = pd.Series(np.zeros(adt_pheno.shape[0]), index=adt_pheno.index)

In [89]:
time_now = datetime.datetime.now()
meanSys = np.zeros((adt_pheno.shape[0],1))
diffSys = np.zeros((adt_pheno.shape[0],1))
meanDias= np.zeros((adt_pheno.shape[0],1))
diffDias= np.zeros((adt_pheno.shape[0],1))
NonExistCounter = 0
for i in range(adt_pheno.shape[0]):
    if i % 1000 == 0:
        print(i)
    if adt_phenoNP[i][0] not in ruidDict:
        NonExistCounter +=1
        continue
    ruidIndexList = ruidDict[adt_phenoNP[i][0]]
    closestDayDiff = 1e5
    indexClosestDayDiff  = 0
    furthestDayDiff = -1e5
    indexFurthestDayDiff = 0
    counterInBetweenDate = 0
    for j in range(len(ruidIndexList)):
        try:
            date = datetime.datetime.strptime(bpNP[ruidIndexList[j]][3], '%m/%d/%y')
        except:
            continue
        if not isinstance(date, datetime.datetime):
            continue
        start = adt_phenoNP[i][1]
        end =  adt_phenoNP[i][2]
        if start <= date <= end:
            counterInBetweenDate +=1
            meanSys[i] = (meanSys[i]*(counterInBetweenDate-1) + bpNP[ruidIndexList[j]][1])/counterInBetweenDate
            meanDias[i] = (meanDias[i]*(counterInBetweenDate-1) + bpNP[ruidIndexList[j]][2])/counterInBetweenDate
            dayDiff = (date - start)
            dayDiff = dayDiff.days
            if dayDiff < closestDayDiff:
                closestDayDiff = dayDiff
                indexClosestDayDiff = ruidIndexList[j]
            if dayDiff > furthestDayDiff:
                furthestDayDiff = dayDiff
                indexFurthestDayDiff = ruidIndexList[j]
    if indexClosestDayDiff == 0 or indexFurthestDayDiff == 0:
        #print('Skip',i)
        continue
    diffSys[i] = bpNP[indexFurthestDayDiff][1] - bpNP[indexClosestDayDiff][1]
    diffDias[i] =  bpNP[indexFurthestDayDiff][2] - bpNP[indexClosestDayDiff][2]
adt_pheno.meanSys = meanSys
adt_pheno.diffSys = diffSys
adt_pheno.meanDias = meanDias
adt_pheno.diffDias = diffDias
diff = datetime.datetime.now() - time_now
diff

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000


datetime.timedelta(0, 373, 243750)

In [91]:
bmi = pd.read_csv('Data/FONNESBECK_BMI_20151202.csv',quoting=csv.QUOTE_NONE)
bmiNP = bmi.values

In [92]:
ruidDict = dict()
for i in range(len(bmiNP)):
    ruid = bmiNP[i][0]
    if ruid not in ruidDict:
        ruidList = []
        ruidList.append(i)
        ruidDict[ruid] = ruidList
    else:
        ruidList = ruidDict[ruid]
        ruidList.append(i)
        ruidDict[ruid] = ruidList

In [93]:
adt_pheno['meanBMI'] = pd.Series(np.zeros(adt_pheno.shape[0]), index=adt_pheno.index)
adt_pheno['diffBMI'] = pd.Series(np.zeros(adt_pheno.shape[0]), index=adt_pheno.index)
adt_pheno['meanBMI_H'] = pd.Series(np.zeros(adt_pheno.shape[0]), index=adt_pheno.index)
adt_pheno['diffBMI_H'] = pd.Series(np.zeros(adt_pheno.shape[0]), index=adt_pheno.index)
adt_pheno['meanBMI_W'] = pd.Series(np.zeros(adt_pheno.shape[0]), index=adt_pheno.index)
adt_pheno['diffBMI_W'] = pd.Series(np.zeros(adt_pheno.shape[0]), index=adt_pheno.index)
adt_pheno['Pregnancy'] = pd.Series(np.zeros(adt_pheno.shape[0]), index=adt_pheno.index)

In [94]:
time_now = datetime.datetime.now()
meanBMI = np.zeros((adt_pheno.shape[0],1))
diffBMI = np.zeros((adt_pheno.shape[0],1))
meanBMI_H= np.zeros((adt_pheno.shape[0],1))
diffBMI_H= np.zeros((adt_pheno.shape[0],1))
meanBMI_W= np.zeros((adt_pheno.shape[0],1))
diffBMI_W= np.zeros((adt_pheno.shape[0],1))
Pregnancy = np.zeros((adt_pheno.shape[0],1))
nonExistCounter = 0
for i in range(adt_pheno.shape[0]):
    if i % 1000 ==0:
        print(i)
    if adt_phenoNP[i][0] not in ruidDict:
        nonExistCounter +=1
        continue
    ruidIndexList = ruidDict[adt_phenoNP[i][0]]
    closestDayDiff = 1e5
    indexClosestDayDiff  = 0
    furthestDayDiff = -1e5
    indexFurthestDayDiff = 0
    counterInBetweenDate = 0
    for j in range(len(ruidIndexList)):
        try:
            date = datetime.datetime.strptime(bmiNP[ruidIndexList[j]][2], '%m/%d/%Y')
        except:
            continue
        if not isinstance(date, datetime.datetime):
            continue
        start = adt_phenoNP[i][1]
        end =  adt_phenoNP[i][2]
        if start <= date <= end:
            counterInBetweenDate +=1
            meanBMI[i] = (meanBMI[i]*(counterInBetweenDate-1) + bmiNP[ruidIndexList[j]][1])/counterInBetweenDate
            meanBMI_W[i] = (meanBMI_W[i]*(counterInBetweenDate-1) + bmiNP[ruidIndexList[j]][3])/counterInBetweenDate
            meanBMI_H[i] = (meanBMI_H[i]*(counterInBetweenDate-1) + bmiNP[ruidIndexList[j]][4])/counterInBetweenDate
            Pregnancy[i] = Pregnancy[i] or bmiNP[ruidIndexList[j]][5]
            dayDiff = (date - start)
            dayDiff = dayDiff.days
            if dayDiff < closestDayDiff:
                closestDayDiff = dayDiff
                indexClosestDayDiff = ruidIndexList[j]
            if dayDiff > furthestDayDiff:
                furthestDayDiff = dayDiff
                indexFurthestDayDiff = ruidIndexList[j]
    if indexClosestDayDiff == 0 or indexFurthestDayDiff == 0:
        #print('Skip',i)
        continue
    diffBMI[i] = bmiNP[indexFurthestDayDiff][1] - bmiNP[indexClosestDayDiff][1]
    diffBMI_W[i] =  bmiNP[indexFurthestDayDiff][3] - bmiNP[indexClosestDayDiff][3]
    diffBMI_H[i] = bmiNP[indexFurthestDayDiff][4] - bmiNP[indexClosestDayDiff][4]
adt_pheno['meanBMI'] = meanBMI
adt_pheno['diffBMI'] = diffBMI
adt_pheno['meanBMI_H'] = meanBMI_H
adt_pheno['diffBMI_H'] = diffBMI_H
adt_pheno['meanBMI_W'] = meanBMI_W
adt_pheno['diffBMI_W'] = diffBMI_W
adt_pheno['Pregnancy'] = Pregnancy
diff = datetime.datetime.now() - time_now
diff

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000


datetime.timedelta(0, 81, 147983)

In [96]:
egfr = pd.read_csv('Data/FONNESBECK_EGFR_20151202.csv',quoting=csv.QUOTE_NONE)
egfrNP = egfr.values

In [97]:
ruidDict = dict()
for i in range(len(egfrNP)):
    ruid = egfrNP[i][0]
    if ruid not in ruidDict:
        ruidList = []
        ruidList.append(i)
        ruidDict[ruid] = ruidList
    else:
        ruidList = ruidDict[ruid]
        ruidList.append(i)
        ruidDict[ruid] = ruidList

In [98]:
adt_pheno['meanEGFR'] = pd.Series(np.zeros(adt_pheno.shape[0]), index=adt_pheno.index)
adt_pheno['diffEGFR'] = pd.Series(np.zeros(adt_pheno.shape[0]), index=adt_pheno.index)

In [99]:
time_now = datetime.datetime.now()
meanEGFR = np.zeros((adt_pheno.shape[0],1))
diffEGFR = np.zeros((adt_pheno.shape[0],1))
nonExistCounter = 0
for i in range(adt_pheno.shape[0]):
    if i % 1000 == 0:
        print(i)
    if adt_phenoNP[i][0] not in ruidDict:
        nonExistCounter +=1
        continue
    ruidIndexList = ruidDict[adt_phenoNP[i][0]]
    closestDayDiff = 1e5
    indexClosestDayDiff  = 0
    furthestDayDiff = -1e5
    indexFurthestDayDiff = 0
    counterInBetweenDate = 0
    for j in range(len(ruidIndexList)):
        try:
            date = datetime.datetime.strptime(egfrNP[ruidIndexList[j]][2], '%m/%d/%Y')
        except:
            continue
        if not isinstance(date, datetime.datetime):
            continue
        start = adt_phenoNP[i][1]
        end =  adt_phenoNP[i][2]
        if start <= date <= end:
            #print('here')
            counterInBetweenDate +=1
            meanEGFR[i] = (meanEGFR[i]*(counterInBetweenDate-1) + egfrNP[ruidIndexList[j]][1])/counterInBetweenDate
            dayDiff = (date - start)
            dayDiff = dayDiff.days
            if dayDiff < closestDayDiff:
                closestDayDiff = dayDiff
                indexClosestDayDiff = ruidIndexList[j]
            if dayDiff > furthestDayDiff:
                furthestDayDiff = dayDiff
                indexFurthestDayDiff = ruidIndexList[j]
    if indexClosestDayDiff == 0 or indexFurthestDayDiff == 0:
        #print('Skip',i)
        continue
    diffEGFR[i] = egfrNP[indexFurthestDayDiff][1] - egfrNP[indexClosestDayDiff][1]
adt_pheno['meanEGFR'] = meanEGFR
adt_pheno['diffEGFR'] = diffEGFR
diff = datetime.datetime.now() - time_now
diff

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000


datetime.timedelta(0, 126, 138842)

In [106]:
a = adt_pheno.readmittedIn30.as_matrix()
type(a)
np.sum(a)

8161.0

In [101]:
adt_pheno.head(10)

,RUID,Admission_date,SRV_CODE,CHIEF_COMPLAINT,DISCHARGE_DATE,Sex,DOB,Race,AGE,ICD9_0,...,diffDias,meanBMI,diffBMI,meanBMI_H,diffBMI_H,meanBMI_W,diffBMI_W,Pregnancy,meanEGFR,diffEGFR
0,50135262,2007-02-08,ORT,R FEMUR FX,2007-02-12,F,1949-09-20,W,57,3,...,-19,0.000000,0.00,0.000000,0,0.000000,0.00,0,123.678915,0.00000
1,50135262,2011-02-11,NEU,SEIZURES;HYPERGLYCEMIA,2011-02-23,F,1949-09-20,W,61,0,...,-43,39.270833,17.34,175.000000,0,120.265833,53.10,0,91.121341,27.60827
2,50135262,2008-02-24,GMD,RECTAL BLEEDING,2008-02-28,F,1949-09-20,W,58,2,...,-9,47.140000,0.00,170.180000,0,136.530000,0.00,0,77.374492,-21.44846
3,50135262,2008-04-12,CAR,CP,2008-04-13,F,1949-09-20,W,58,1,...,12,47.580000,0.00,170.180000,0,137.800000,0.00,0,84.358415,0.00000
4,50135262,2012-05-23,CAR,SINUS NODE DYSFUCTION;SYNCOPE,2012-05-27,F,1949-09-20,W,62,2,...,19,44.043333,-1.75,170.195556,0,127.578889,-5.06,0,59.974333,-2.13620
5,50135262,2007-08-03,CAR,CP,2007-08-06,F,1949-09-20,W,57,6,...,-9,45.092500,0.56,170.180000,0,130.600000,1.60,0,89.548814,-4.33354
6,50135262,2007-08-28,EMR,CHEST PAIN,2007-08-29,F,1949-09-20,W,57,1,...,9,42.606667,12.53,170.180000,0,123.400000,36.30,0,107.449655,0.00000
7,50135262,2012-09-15,ORT,FEMUR FX,2012-09-22,F,1949-09-20,W,62,3,...,-57,0.000000,0.00,0.000000,0,0.000000,0.00,0,75.721047,31.12508
8,50135262,2010-10-28,NEU,SEIZURE,2010-10-29,F,1949-09-20,W,61,0,...,-7,0.000000,0.00,0.000000,0,0.000000,0.00,0,101.375045,-9.21008
9,50135262,2014-11-15,NEU,345.41 REFRACTORY SEIZURES,2014-11-18,F,1949-09-20,W,65,3,...,-2,48.340000,0.00,170.180000,0,140.000000,0.00,0,0.000000,0.00000


In [107]:
adt_pheno.to_csv('adt_Ahmet.csv')